# Graph Network

In [ ]:
print(torch.__version__)
print(torch.version.cuda)

In [7]:
!pip install torch-scatter==latest+cu102 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-sparse==latest+cu102 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-cluster==latest+cu102 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-spline-conv==latest+cu102 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0.html
     |████████████████████████████████| 11.6 MB 135 kB/s eta 0:00:01
  Attempting uninstall: torch-scatter
    Found existing installation: torch-scatter 2.0.5
    Uninstalling torch-scatter-2.0.5:
      Successfully uninstalled torch-scatter-2.0.5
Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0.html
     |████████████████████████████████| 23.0 MB 101 kB/s eta 0:00:01
  Attempting uninstall: torch-sparse
    Found existing installation: torch-sparse 0.6.7
    Uninstalling torch-sparse-0.6.7:
      Successfully uninstalled torch-sparse-0.6.7
Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0.html
     |████████████████████████████████| 20.3 MB 152 kB/s eta 0:00:01
  Attempting uninstall: torch-cluster
    Found existing installation: torch-cluster 1.5.7
    Uninstalling torch-cluster-1.5.7:
      Successfully uninstalled torch-cluster-1.5.7
Looking in links: https://pytorch-geometric.com/wh

## Brainstorming

图神经网络与图卷积网络 https://zhuanlan.zhihu.com/p/70028587 图监督学习的统一分析框架是：

Aggregator:
$$
m_v^{t+1} = \sum_{w \in N_v} M_t(h_v^t, h_w^t, e_{vw})
$$

Updater:
$$
h_v^{t+1} = U_t(h_v^t, m_v^{t+1})
$$

#  Quick Start


## GCN 网络论文分类数据

### 从 Cora 出发 with GCN

Cora：一个根据科学论文之间相互引用关系而构建的Graph数据集合，论文分为7类：Genetic_Algorithms，Neural_Networks，Probabilistic_Methods，Reinforcement_Learning，Rule_Learning，Theory，共2708篇；

In [6]:
import torch
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

ModuleNotFoundError: No module named 'torch_sparse'

In [5]:
# !pip install torch_sparse # 安装有问题

  Using cached https://files.pythonhosted.org/packages/db/d1/968436c29959c321740ee95f781c961f12b2d23f0ecdbdaaf3ccf64ddc94/torch_sparse-0.6.6.tar.gz
  ERROR: Command errored out with exit status 1:
   command: /home/yons/anaconda3/envs/torch/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-_leqep1z/torch-sparse/setup.py'"'"'; __file__='"'"'/tmp/pip-install-_leqep1z/torch-sparse/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-96z0urld --python-tag cp36
       cwd: /tmp/pip-install-_leqep1z/torch-sparse/
  Complete output (83 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.6
  creating build/lib.linux-x86_64-3.6/torch_sparse
  copying torch_sparse/narrow.py -> build/lib.linux-x86_64-3.6/torch_sparse
  copying torch_sparse/matmu

In [2]:
#数据集加载
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='./tmp/Cora', name='Cora')
len(dataset), dataset.num_classes, dataset.num_node_features

ModuleNotFoundError: No module named 'torch_geometric'

In [21]:
!tree ./tmp/Cora/

./tmp/Cora/
├── processed
│   ├── data.pt
│   ├── pre_filter.pt
│   └── pre_transform.pt
└── raw
    ├── ind.cora.allx
    ├── ind.cora.ally
    ├── ind.cora.graph
    ├── ind.cora.test.index
    ├── ind.cora.tx
    ├── ind.cora.ty
    ├── ind.cora.x
    └── ind.cora.y

2 directories, 11 files


In [7]:
#网络定义
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

NameError: name 'GCNConv' is not defined

网络的输入输入是什么？

2708篇论文，1403 个单词。


In [23]:
data.x.shape, data.y.shape, data.edge_index.shape

(torch.Size([2708, 1433]), torch.Size([2708]), torch.Size([2, 10556]))

In [24]:
#网络训练
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

需要看看输出是什么？

In [6]:
print(torch.version.cuda)
print(torch.__version__)

10.2
1.6.0


In [25]:
out.shape

torch.Size([2708, 7])

In [26]:
#测试
model.eval()
_, pred = model(data).max(dim=1)
correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.7970


### 快速 Citeseer


Citeseer：一个论文之间引用信息数据集，论文分为6类：Agents、AI、DB、IR、ML和HCI，共包含3312篇论文；

In [10]:
import torch
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

#数据集加载
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/tmp/Citeseer', name='Citeseer')

In [11]:
?Planetoid

Init signature: Planetoid(root, name, transform=None, pre_transform=None)
Docstring:     
The citation network datasets "Cora", "CiteSeer" and "PubMed" from the
`"Revisiting Semi-Supervised Learning with Graph Embeddings"
<https://arxiv.org/abs/1603.08861>`_ paper.
Nodes represent documents and edges represent citation links.
Training, validation and test splits are given by binary masks.

Args:
    root (string): Root directory where the dataset should be saved.
    name (string): The name of the dataset (:obj:`"Cora"`,
        :obj:`"CiteSeer"`, :obj:`"PubMed"`).
    transform (callable, optional): A function/transform that takes in an
        :obj:`torch_geometric.data.Data` object and returns a transformed
        version. The data object will be transformed before every access.
        (default: :obj:`None`)
    pre_transform (callable, optional): A function/transform that takes in
        an :obj:`torch_geometric.data.Data` object and returns a
        transformed version. The da

In [12]:
#网络定义
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 32)
        self.conv2 = GCNConv(32, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

#网络训练
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

#测试
model.eval()
_, pred = model(data).max(dim=1)
correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.6860


## Graph Attention Network

我需要快速实现 Graph Attention Network. 官方教程中有。。。。



官方教程中有一个显卡使用的问题 RuntimeError: CUDA error: invalid device function， 我绕过去了。see https://github.com/rusty1s/pytorch_geometric/blob/master/examples/gat.py

In [9]:
import os.path as osp

import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GATConv

In [15]:
dataset = 'Cora'
dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [29]:
data  = dataset[0]
data.train_mask

tensor([ True,  True,  True,  ..., False, False, False])

path = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', dataset)
path = osp.join('./tmp', dataset)
dataset = Planetoid(path, dataset, T.NormalizeFeatures())
data = dataset[0]

In [22]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GATConv(dataset.num_features, 8, heads=8, dropout=0.6)
        # On the Pubmed dataset, use heads=8 in conv2.
        self.conv2 = GATConv(
            8 * 8, dataset.num_classes, heads=1, concat=True, dropout=0.6)

    def forward(self):
        x = F.dropout(data.x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, data.edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, data.edge_index)
        return F.log_softmax(x, dim=1)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

In [23]:
data.y, device.index, torch.cuda.is_available()

(tensor([3, 4, 4,  ..., 3, 3, 3]), None, True)

In [24]:
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()

def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

for epoch in range(1, 201):
    train()
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    print(log.format(epoch, *test()))

Epoch: 001, Train: 0.5000, Val: 0.4140, Test: 0.4090
Epoch: 002, Train: 0.6643, Val: 0.5360, Test: 0.5070
Epoch: 003, Train: 0.7357, Val: 0.5860, Test: 0.5540
Epoch: 004, Train: 0.7857, Val: 0.6020, Test: 0.5860
Epoch: 005, Train: 0.8429, Val: 0.6440, Test: 0.6470
Epoch: 006, Train: 0.8857, Val: 0.6740, Test: 0.6850
Epoch: 007, Train: 0.9071, Val: 0.6960, Test: 0.7120
Epoch: 008, Train: 0.9143, Val: 0.7140, Test: 0.7320
Epoch: 009, Train: 0.9286, Val: 0.7300, Test: 0.7490
Epoch: 010, Train: 0.9357, Val: 0.7400, Test: 0.7560
Epoch: 011, Train: 0.9429, Val: 0.7520, Test: 0.7660
Epoch: 012, Train: 0.9500, Val: 0.7500, Test: 0.7660
Epoch: 013, Train: 0.9500, Val: 0.7500, Test: 0.7700
Epoch: 014, Train: 0.9500, Val: 0.7520, Test: 0.7660
Epoch: 015, Train: 0.9500, Val: 0.7520, Test: 0.7640
Epoch: 016, Train: 0.9500, Val: 0.7540, Test: 0.7630
Epoch: 017, Train: 0.9500, Val: 0.7520, Test: 0.7650
Epoch: 018, Train: 0.9500, Val: 0.7480, Test: 0.7690
Epoch: 019, Train: 0.9571, Val: 0.7520, Test: 

## 通俗理解

需要一些关于图网络的通俗理解

- PyTorch Geometric 攻略

在 PyTorch Geometric 中，一个图被定义为g=(X,(I,E))，其中X表示节点的特征矩阵，N为节点的个数，F为每个节点的特征数；用I,E这种元组形式表示图的稀疏邻接矩阵，I为边的索引，E为D维的边特征。用于模型的图（graph）数据包括对象（nodes）及成对对象之间的关系（edges）组成。用于 PyTorch Geometric 中的每个图都是一个 torch_geometric.data.Data 类型的实例，其属性有：

- data.x：节点特征矩阵，形状为 [num_nodes, num_node_features]。
- data.edge_index：COO 格式的图的边关系，形状为 [2, num_edges]，类型为 torch.long
- data.edge_attr：边特征矩阵，形状为[num_edges, num_edge_features]
- data.y：针对训练的目标可能具有不同的形状
- data.pos：节点的位置矩阵，形状为[num_nodes, num_dimensions]

Data 对象不是必须有上面所有的这些属性，也不是只能有这些属性。比如，我们可以通过data.face进行扩展，用一个张量（tensor）来保存一个3D网格的三元链接关系，形状为 [3, num_faces]，类型为 torch.long。


- 官方理解 [Creating Message Passing Networks](https://github.com/rusty1s/pytorch_geometric/blob/master/docs/source/notes/create_gnn.rst)


Generalizing the convolution operator to irregular domains is typically expressed as a *neighborhood aggregation* or *message passing* scheme. With $\mathbf{x}^{(k-1)}_i \in \mathbb{R}^F$ denoting node features of node $i$ in layer $(k-1)$ and $\mathbf{e}_{i,j} \in \mathbb{R}^D$ denoting (optional) edge features from node $i$ to node $j$, message passing graph neural networks can be described as

$$
\mathbf{x}_i^{(k)} = \gamma^{(k)} \left( \mathbf{x}_i^{(k-1)}, \square_{j \in \mathcal{N}(i)} \, \phi^{(k)}\left(\mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)},\mathbf{e}_{i,j}\right) \right),
$$

where $\square$ denotes a differentiable, permutation invariant function, e.g., sum, mean or max, and $\gamma$ and $\phi$ denote differentiable functions such as MLPs (Multi Layer Perceptrons).

# End to End

# Zero to All


## 重要资料

- GCN作者 slide http://tkipf.github.io/misc/SlidesCambridge.pdf

- 清华 Peng Cui http://www.cips-cl.org/static/CCL2019/downloads/tutorialsPPT/02.pdf

- https://github.com/rusty1s/pytorch_geometric

PyTorch Geometric (PyG) is a geometric deep learning extension library for PyTorch.

It consists of various methods for deep learning on graphs and other irregular structures, also known as geometric deep learning, from a variety of published papers. In addition, it consists of an easy-to-use mini-batch loader for many small and single giant graphs, multi gpu-support, a large number of common benchmark datasets (based on simple interfaces to create your own), and helpful transforms, both for learning on arbitrary graphs as well as on 3D meshes or point clouds. 

- https://pytorch-geometric.readthedocs.io/en/latest/index.html

PyTorch Geometric user document.



## 图卷积层

我们需要指导图卷积层的数学原理和实现。


In [18]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')  # "Add" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: 增加自连接到邻接矩阵
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: 对节点的特征矩阵进行线性变换
        x = self.lin(x)

        # Step 3-5: Start propagating messages.
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j, edge_index, size):
        # x_j has shape [E, out_channels]

        # Step 3: Normalize node features.
        row, col = edge_index
        deg = degree(row, size[0], dtype=x_j.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]

        # Step 5: Return new node embeddings.
        return aggr_out